# 야놀자 체크인, 체크아웃 누락 크롤링

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(0.4)  # 페이지 로딩 대기
# search_input = driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input')
# time.sleep(0.8)  # 페이지 로딩 대기

# search_input.send_keys('괴산 더진펜션')
# search_input.send_keys(Keys.ENTER)
# time.sleep(1.2)
# driver.find_element(By.XPATH , value = '/html/body/div[1]/div/div[2]/div/div[1]/a/div/div[2]/div[1]/p').click()

In [ ]:
import pandas as pd
df_체크 = pd.read_csv('0807체크인아웃누락v1.csv')

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(0.4)
all_product_CD = all_product_CD['LDGS_CD'].tolist()

def 야놀자(all_product_CD):
    data = []

    for idx, pid in enumerate(all_product_CD):
        url = f'https://place-site.yanolja.com/places/{pid}'
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')



        prev_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # 아래로 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)  # 로딩 대기

            # 스크롤 후 높이 측정
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # 더 이상 스크롤할 곳 없으면 종료
            if new_height == prev_height:
                break
            prev_height = new_height



            time.sleep(1.2)
            driver.find_element(By.XPATH , value = '//*[@id="domestic-pdp-info"]/div[5]/div/div/div[3]/div/button').click()



            # 체크인
            CHIN_TIME = soup.find('div', class_='css-1yzvfwv')
            CHIN_TIME = CHIN_TIME.text if CHIN_TIME else '정보 없음'


            # 체크아웃
            CHOT_TIME = soup.find('span', class_='css-1yzvfwv')
            CHOT_TIME = CHOT_TIME.text if CHOT_TIME else '정보 없음'




In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd

# === 1. 드라이버 설정 (봇 감지 우회) ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

# === 2. 진입용 페이지 오픈 (쿠키 설정 등 초기화 목적) ===
url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(0.4)

# === 3. 상품 ID 리스트 불러오기 ===
all_product_CD = pd.read_csv('체크인아웃누락.csv')
all_product_CD['LDGS_CD'] = all_product_CD['LDGS_CD'].astype(str)
all_product_CD = all_product_CD['LDGS_CD'].tolist()

# === 4. 크롤링 함수 정의 ===
def 야놀자(driver, all_product_CD):
    data = []

    for idx, pid in enumerate(all_product_CD):
        print(f"[{idx+1}] 숙소 ID {pid} 페이지 접속 중...")
        place_url = f'https://place-site.yanolja.com/places/{pid}'
        driver.get(place_url)
        time.sleep(2)

        # === 스크롤 다운 ===
        try:
            prev_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.5)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == prev_height:
                    break
                prev_height = new_height
        except:
            print("스크롤 실패")

        # === 버튼 클릭 (필요 시) ===
        try:
            btn = driver.find_element(By.XPATH, '//*[@id="domestic-pdp-info"]/div[5]/div/div/div[3]/div/button')
            btn.click()
            time.sleep(1)
        except:
            pass  # 버튼 없을 수도 있음

        # === 데이터 추출 ===
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        checkin = soup.find('div', class_='css-1yzvfwv')
        checkout = soup.find('span', class_='css-1yzvfwv')

        checkin = checkin.text.strip() if checkin else '정보 없음'
        checkout = checkout.text.strip() if checkout else '정보 없음'

        data.append({
            'LDGS_CD': pid,
            '체크인': checkin,
            '체크아웃': checkout
        })

    return data

# === 5. 실행 및 출력 ===
result = 야놀자(driver, all_product_CD)

# 콘솔 출력
for item in result:
    print(item)

# CSV 저장
df = pd.DataFrame(result)
# df.to_csv("야놀자_체크인_체크아웃.csv", index=False, encoding='utf-8-sig')
# print("CSV 저장 완료.")
df
# 드라이버 종료 (필요 시)
# driver.quit()


In [ ]:
df_체크 = pd.read_csv('0807체크인아웃누락v1.csv')
df_체크['LDGMNT_TY_NM'].value_counts()

In [ ]:
# LDGMNT_TY_NM Pension만 추출
pension_only = all_product_CD[all_product_CD['LDGMNT_TY_NM'] == 'Hotel']
pension_only

## 펜션 체크인 체크아웃

- 10개만 확인

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 및 상위 10개 추출 ===
df = pd.read_csv('체크인아웃누락.csv')
df = df[df['LDGMNT_TY_NM'] == 'Pension']
ids = df['LDGS_CD'].astype(str).tolist()[:10]

# === 4. 크롤링 함수 ===
def 야놀자_체크인아웃(driver, id_list):
    result = []

    for idx, pid in enumerate(id_list):
        print(f"\n[{idx+1}] 숙소 ID {pid} 접속 중...")
        driver.get(f"https://place-site.yanolja.com/places/{pid}")
        time.sleep(2.5)

        # === 전체보기 버튼 클릭 ===
        try:
            btn = driver.find_element(By.CSS_SELECTOR, 'button.css-ff1hhr')
            btn.click()
            print("✅ 전체보기 클릭 성공")
            time.sleep(1.5)
        except:
            print("❌ 전체보기 버튼 없음")

        # === iframe 내부 스크롤 ===
        try:
            scroll_target = driver.find_element(By.CSS_SELECTOR, 'div.css-1ulzvpi')
            for _ in range(5):
                ActionChains(driver).move_to_element(scroll_target).click().perform()
                driver.execute_script("arguments[0].scrollTop += 500;", scroll_target)
                time.sleep(1)
            print("✅ 스크롤 완료")
        except Exception as e:
            print("❌ 스크롤 실패:", e)

        # === 체크인/체크아웃 정보 추출 ===
        checkin, checkout = "정보 없음", "정보 없음"
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//h3[contains(text(), '체크인')]/following-sibling::div"))
            )
            checkin_raw = driver.find_element(By.XPATH, "//h3[contains(text(), '체크인')]/following-sibling::div")
            checkout_raw = driver.find_element(By.XPATH, "//h3[contains(text(), '체크아웃')]/following-sibling::div")

            # ✅ text 대신 textContent
            checkin = checkin_raw.get_attribute("textContent").strip()
            checkout = checkout_raw.get_attribute("textContent").strip()
            print(f"✅ 체크인: {checkin} / 체크아웃: {checkout}")
        except Exception as e:
            print("⚠️ 체크인/체크아웃 추출 실패:", e)

        result.append({
            "LDGS_CD": pid,
            "체크인": checkin,
            "체크아웃": checkout
        })

    return pd.DataFrame(result)

# === 5. 실행 ===
df_result = 야놀자_체크인아웃(driver, ids)

# === 6. 출력 및 저장 ===
print("\n📦 최종 결과:")
print(df_result)
df_result.to_csv("야놀자_체크인아웃_펜션.csv", index=False)


- 최종 코드

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.5)

# === 3. 숙소 ID 불러오기 및 필터링 ===
df_체크 = pd.read_csv('0807체크인아웃누락v1.csv')
df_펜션 = df_체크[df_체크['LDGMNT_TY_NM'] == 'Pension']
df_펜션['숙소ID'] = df_펜션['숙소ID'].astype(str)

# === 4. 이미 수집된 숙소 제거 (중단 후 재시작용) ===
completed_ids = []
if os.path.exists("야놀자_체크인아웃_펜션.csv"):
    completed_df = pd.read_csv("야놀자_체크인아웃_펜션.csv")
    completed_ids = completed_df['숙소ID'].astype(str).tolist()

# 수집 대상 ID 리스트
target_ids = df_펜션[~df_펜션['숙소ID'].isin(completed_ids)]['숙소ID'].tolist()
print(f"\n✅ 전체 대상: {len(df_펜션)}개 | 남은 수집 대상: {len(target_ids)}개\n")

# === 5. 크롤링 함수 정의 ===
def 야놀자_체크인아웃(driver, id_list, save_every=50):
    result = []
    fail_log = []

    for idx, pid in enumerate(id_list):
        print(f"\n[{idx+1}/{len(id_list)}] 숙소 ID {pid} 접속 중...")
        driver.get(f"https://place-site.yanolja.com/places/{pid}")
        time.sleep(2.5)

        # 전체보기 버튼 클릭
        try:
            btn = driver.find_element(By.CSS_SELECTOR, 'button.css-ff1hhr')
            btn.click()
            time.sleep(1.5)
            print("✅ 전체보기 클릭 성공")
        except:
            print("❌ 전체보기 버튼 없음")

        # iframe 스크롤
        try:
            scroll_target = driver.find_element(By.CSS_SELECTOR, 'div.css-1ulzvpi')
            for _ in range(5):
                ActionChains(driver).move_to_element(scroll_target).click().perform()
                driver.execute_script("arguments[0].scrollTop += 500;", scroll_target)
                time.sleep(1)
            print("✅ 스크롤 완료")
        except Exception as e:
            print("❌ 스크롤 실패:", e)

        # 체크인/체크아웃 추출
        checkin, checkout = "정보 없음", "정보 없음"
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//h3[contains(text(), '체크인')]/following-sibling::div"))
            )
            checkin_raw = driver.find_element(By.XPATH, "//h3[contains(text(), '체크인')]/following-sibling::div")
            checkout_raw = driver.find_element(By.XPATH, "//h3[contains(text(), '체크아웃')]/following-sibling::div")
            checkin = checkin_raw.get_attribute("textContent").strip()
            checkout = checkout_raw.get_attribute("textContent").strip()
            print(f"✅ 체크인: {checkin} / 체크아웃: {checkout}")
        except Exception as e:
            print("⚠️ 체크인/체크아웃 추출 실패:", e)
            fail_log.append({"LDGS_CD": pid, "이유": "체크인/체크아웃 없음"})

        result.append({
            "LDGS_CD": pid,
            "체크인": checkin,
            "체크아웃": checkout
        })

        # === 50개 단위로 저장 ===
        if (idx + 1) % save_every == 0 or (idx + 1) == len(id_list):
            temp_df = pd.DataFrame(result)
            if os.path.exists("야놀자_체크인아웃_펜션.csv"):
                temp_df.to_csv("야놀자_체크인아웃_펜션.csv", mode='a', header=False, index=False)
            else:
                temp_df.to_csv("야놀자_체크인아웃_펜션.csv", index=False)
            print(f"\n💾 저장 완료 (누적 {idx+1}개)\n")
            result.clear()  # 메모리 절약

    # 실패 로그 저장
    if fail_log:
        pd.DataFrame(fail_log).to_csv("야놀자_크롤링_실패로그.csv", index=False)
        print(f"🚨 실패 로그 {len(fail_log)}건 저장 완료")

# === 6. 실행 ===
야놀자_체크인아웃(driver, target_ids)

print("\n🎉 전체 크롤링 완료")


- 누락으로 재크롤링

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options, version_main=138)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 및 상위 10개 추출 ===
df = pd.read_csv('야놀자_크롤링_실패로그.csv')
# df = df[df['LDGMNT_TY_NM'] == 'Pension']
ids = df['LDGS_CD'].astype(str).tolist()

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def 야놀자_체크인아웃(driver, id_list):
    result = []

    for idx, pid in enumerate(id_list):
        print(f"\n[{idx+1}] 숙소 ID {pid} 접속 중...")

        try:
            driver.get(f"https://place-site.yanolja.com/places/{pid}")
            time.sleep(2.5)

            # === 스크롤 내리기
            for _ in range(3):
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(0.8)

            # === 체크인/체크아웃 추출
            checkin_time = "정보 없음"
            checkout_time = "정보 없음"

            try:
                WebDriverWait(driver, 8).until(
                    EC.presence_of_all_elements_located((By.TAG_NAME, "span"))
                )
                spans = driver.find_elements(By.TAG_NAME, "span")
                span_texts = [s.text.strip() for s in spans if s.text.strip()]
                print("📌 span_texts:", span_texts[:10])

                for i in range(len(span_texts) - 1):
                    try:
                        if "체크인" in span_texts[i] and ":" in span_texts[i + 1]:
                            checkin_time = span_texts[i + 1]
                        if "체크아웃" in span_texts[i] and ":" in span_texts[i + 1]:
                            checkout_time = span_texts[i + 1]
                        if checkin_time != "정보 없음" and checkout_time != "정보 없음":
                            break
                    except IndexError:
                        continue

            except Exception as inner_e:
                print(f"❌ [내부 파싱 오류] 숙소 ID {pid}: {inner_e}")

        except Exception as outer_e:
            print(f"❌ [페이지 접속 오류] 숙소 ID {pid}: {outer_e}")
            checkin_time = "접속 오류"
            checkout_time = "접속 오류"

        # ✅ 무조건 결과 저장
        result.append({
            "LDGS_CD": pid,
            "체크인": checkin_time,
            "체크아웃": checkout_time
        })

    return pd.DataFrame(result)

df_result = 야놀자_체크인아웃(driver, ids)
df_result.to_csv("야놀자_체크인아웃_펜션누락.csv", index=False)
df_result.head()

## 게하 체크인 체크아웃


In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options, version_main=138)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 및 상위 10개 추출 ===
df = pd.read_csv('0807체크인아웃누락v1.csv')
df = df[df['LDGMNT_TY_NM'] == 'GuestHouse']
ids = df['숙소ID'].astype(str).tolist()



In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def 야놀자_체크인아웃(driver, id_list):
    result = []

    for idx, pid in enumerate(id_list):
        print(f"\n[{idx+1}] 숙소 ID {pid} 접속 중...")

        try:
            driver.get(f"https://place-site.yanolja.com/places/{pid}")
            time.sleep(2.5)

            # === 스크롤 내리기
            for _ in range(3):
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(0.8)

            # === 체크인/체크아웃 추출
            checkin_time = "정보 없음"
            checkout_time = "정보 없음"

            try:
                WebDriverWait(driver, 8).until(
                    EC.presence_of_all_elements_located((By.TAG_NAME, "span"))
                )
                spans = driver.find_elements(By.TAG_NAME, "span")
                span_texts = [s.text.strip() for s in spans if s.text.strip()]
                print("📌 span_texts:", span_texts[:10])

                for i in range(len(span_texts) - 1):
                    try:
                        if "체크인" in span_texts[i] and ":" in span_texts[i + 1]:
                            checkin_time = span_texts[i + 1]
                        if "체크아웃" in span_texts[i] and ":" in span_texts[i + 1]:
                            checkout_time = span_texts[i + 1]
                        if checkin_time != "정보 없음" and checkout_time != "정보 없음":
                            break
                    except IndexError:
                        continue

            except Exception as inner_e:
                print(f"❌ [내부 파싱 오류] 숙소 ID {pid}: {inner_e}")

        except Exception as outer_e:
            print(f"❌ [페이지 접속 오류] 숙소 ID {pid}: {outer_e}")
            checkin_time = "접속 오류"
            checkout_time = "접속 오류"

        # ✅ 무조건 결과 저장
        result.append({
            "LDGS_CD": pid,
            "체크인": checkin_time,
            "체크아웃": checkout_time
        })

    return pd.DataFrame(result)

df_result = 야놀자_체크인아웃(driver, ids)
df_result.to_csv("야놀자_체크인아웃_게스트하우스.csv", index=False)
df_result.head()


## 호텔 체크인 체크아웃

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 ===
df = pd.read_csv('0807체크인아웃누락v1.csv')
df = df[df['LDGMNT_TY_NM'] == 'Hotel']
ids = df['숙소ID'].astype(str).tolist()


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def 야놀자_체크인아웃(driver, id_list):
    result = []

    for idx, pid in enumerate(id_list):
        print(f"\n[{idx+1}/{len(id_list)}] 숙소 ID {pid} 접속 중...")

        try:
            driver.get(f"https://place-site.yanolja.com/places/{pid}")
            time.sleep(2.5)

            # === 스크롤 내리기
            for _ in range(3):
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(0.8)

            # === 체크인/체크아웃 추출
            checkin_time = "정보 없음"
            checkout_time = "정보 없음"

            try:
                WebDriverWait(driver, 8).until(
                    EC.presence_of_all_elements_located((By.TAG_NAME, "span"))
                )
                spans = driver.find_elements(By.TAG_NAME, "span")
                span_texts = [s.text.strip() for s in spans if s.text.strip()]
                print("📌 span_texts:", span_texts[:10])

                for i in range(len(span_texts) - 1):
                    try:
                        if "체크인" in span_texts[i] and ":" in span_texts[i + 1]:
                            checkin_time = span_texts[i + 1]
                        if "체크아웃" in span_texts[i] and ":" in span_texts[i + 1]:
                            checkout_time = span_texts[i + 1]
                        if checkin_time != "정보 없음" and checkout_time != "정보 없음":
                            break
                    except IndexError:
                        continue

            except Exception as inner_e:
                print(f"❌ [내부 파싱 오류] 숙소 ID {pid}: {inner_e}")

        except Exception as outer_e:
            print(f"❌ [페이지 접속 오류] 숙소 ID {pid}: {outer_e}")
            checkin_time = "접속 오류"
            checkout_time = "접속 오류"

        # ✅ 무조건 결과 저장
        result.append({
            "LDGS_CD": pid,
            "체크인": checkin_time,
            "체크아웃": checkout_time
        })

    return pd.DataFrame(result)

df_result = 야놀자_체크인아웃(driver, ids)
df_result.to_csv("야놀자_체크인아웃_호텔.csv", index=False)
df_result.head()

- 누락 값 크롤링

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 ===
df = pd.read_csv('야놀자_체크인아웃_호텔누락.csv')
# df = df[df['LDGMNT_TY_NM'] == 'Hotel']
ids = df['LDGS_CD'].astype(str).tolist()


In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 ===
df = pd.read_csv('야놀자_체크인아웃_호텔누락.csv')
# df = df[df['LDGMNT_TY_NM'] == 'Hotel']
ids = df['LDGS_CD'].astype(str).tolist()


In [ ]:
df_result.info()

## 모텔 체크인아웃

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 ===
df = pd.read_csv('0807체크인아웃누락v1.csv')
df = df[df['LDGMNT_TY_NM'] == 'Motel']
ids = df['숙소ID'].astype(str).tolist()

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def 야놀자_체크인아웃(driver, id_list):
    result = []

    for idx, pid in enumerate(id_list):
        print(f"\n[{idx+1}/{len(id_list)}] 숙소 ID {pid} 접속 중...")

        try:
            driver.get(f"https://place-site.yanolja.com/places/{pid}")
            time.sleep(2.5)

            # === 스크롤 내리기
            for _ in range(3):
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(0.8)

            # === 체크인/체크아웃 추출 (숙박 영역만)
            checkin_time = "정보 없음"
            checkout_time = "정보 없음"

            try:
                # 숙박 관련 요금제 블록 찾기
                containers = driver.find_elements(By.CLASS_NAME, "rate-plan-container")
                found = False

                for container in containers:
                    if "숙박" in container.text:
                        try:
                            spans = container.find_elements(By.TAG_NAME, "span")
                            span_texts = [s.text.strip() for s in spans if s.text.strip()]
                            print("📌 숙박 span_texts:", span_texts[:10])

                            for i in range(len(span_texts) - 1):
                                if "체크인" in span_texts[i] and ":" in span_texts[i + 1]:
                                    checkin_time = span_texts[i + 1]
                                if "체크아웃" in span_texts[i] and ":" in span_texts[i + 1]:
                                    checkout_time = span_texts[i + 1]
                                if checkin_time != "정보 없음" and checkout_time != "정보 없음":
                                    found = True
                                    break
                            if found:
                                break
                        except Exception:
                            continue

            except Exception as inner_e:
                print(f"❌ [내부 파싱 오류] 숙소 ID {pid}: {inner_e}")

        except Exception as outer_e:
            print(f"❌ [페이지 접속 오류] 숙소 ID {pid}: {outer_e}")
            checkin_time = "접속 오류"
            checkout_time = "접속 오류"

        # ✅ 무조건 결과 저장
        result.append({
            "LDGS_CD": pid,
            "체크인": checkin_time,
            "체크아웃": checkout_time
        })

    return pd.DataFrame(result)

# === 실행 ===
df_result = 야놀자_체크인아웃(driver, ids)
df_result.to_csv("야놀자_체크인아웃_모텔.csv", index=False)
df_result.head()


# 야놀자 가격 및 정보 크롤링

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
all_product_CD = pd.read_csv('체크인아웃누락.csv')



options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(0.4)  # 페이지 로딩 대기

driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
time.sleep(0.4)
driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[2]/div/table/tbody/tr[2]/td[7]').click()
time.sleep(0.4)
driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[2]/div/table/tbody/tr[3]/td[1]').click()
time.sleep(0.4)
driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()


# search_input = driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input')
# time.sleep(0.8)  # 페이지 로딩 대기

# search_input.send_keys('괴산 더진펜션')
# search_input.send_keys(Keys.ENTER)
# time.sleep(1.2)
# driver.find_element(By.XPATH , value = '/html/body/div[1]/div/div[2]/div/div[1]/a/div/div[2]/div[1]/p').click()

In [ ]:
import pandas as pd
df1 = pd.read_csv('메인프레임.csv')

## 주중판매가

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options)


In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))


In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_total.csv',
                     failed_path='야놀자_크롤링/failed_log.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result = []
    failed_ids = []
    is_first = not os.path.exists(result_path)

    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click()
            time.sleep(0.3)

            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                rating = driver.find_element(By.CLASS_NAME, 'typography-body-14-bold').text.strip()
            except:
                rating = "없음"

            try:
                review_count = driver.find_element(By.XPATH, '//p[@class="flex items-center justify-start gap-2 pl-2"]/span[2]').text.strip().replace("(", "").replace(")", "")
            except:
                review_count = "없음"

            주중판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주중판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except Exception as e:
                print(f" 숙박가 파싱 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주중판매가": 주중판매가
            })
            print(f" 수집결과 → {result[-1]}")

            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)

            except Exception as e:
                print(f"⚠️ 뒤로가기 실패 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)

            # 실패한 ID를 CSV에 저장
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path,
                mode='a',
                index=False,
                header=not os.path.exists(failed_path),
                encoding='utf-8-sig'
            )

            try:
                driver.quit()
            except:
                pass
            driver = init_driver()
            url = 'https://nol.yanolja.com/search?tab=place'
            driver.get(url)
            time.sleep(random.uniform(2.5, 4.0))
           
                       # ✅ 드라이버 재시작 시 날짜 재선택
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[4]/td[3]').click()
                time.sleep(0.4)                           
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[4]/td[4]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
                continue  # 날짜 클릭도 실패했으면 다음 ID로 넘김
           
            continue

        if len(result) >= save_every:
            df_temp = pd.DataFrame(result)
            df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(df_temp)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    if result:
        df_temp = pd.DataFrame(result)
        df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(df_temp)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")

    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()



In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("메인프레임.csv")
숙소_id_list = df['LDGS_CD'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/숙소정보.csv", index=False)
df_실패.to_csv("야놀자_크롤링/실패목록.csv", index=False)

df_정보.head()


## 1차 주중판매가 누락 상품ID 크롤링

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options)


In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))

In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_주중누락.csv',
                     failed_path='야놀자_크롤링/failed_주중누락.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result = []
    failed_ids = []
    is_first = not os.path.exists(result_path)

    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click()
            time.sleep(0.3)

            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                rating = driver.find_element(By.CLASS_NAME, 'typography-body-14-bold').text.strip()
            except:
                rating = "없음"

            try:
                review_count = driver.find_element(By.XPATH, '//p[@class="flex items-center justify-start gap-2 pl-2"]/span[2]').text.strip().replace("(", "").replace(")", "")
            except:
                review_count = "없음"

            주중판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주중판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except Exception as e:
                print(f" 숙박가 파싱 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주중판매가": 주중판매가
            })
            print(f" 수집결과 → {result[-1]}")

            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)

            except Exception as e:
                print(f"⚠️ 뒤로가기 실패 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)

            # 실패한 ID를 CSV에 저장
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path,
                mode='a',
                index=False,
                header=not os.path.exists(failed_path),
                encoding='utf-8-sig'
            )

            try:
                driver.quit()
            except:
                pass
            driver = init_driver()
            url = 'https://nol.yanolja.com/search?tab=place'
            driver.get(url)
            time.sleep(random.uniform(2.5, 4.0))
           
                       # ✅ 드라이버 재시작 시 날짜 재선택
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                        
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[3]/td[3]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[3]/td[4]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
                continue  # 날짜 클릭도 실패했으면 다음 ID로 넘김
           
            continue

        if len(result) >= save_every:
            df_temp = pd.DataFrame(result)
            df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(df_temp)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    if result:
        df_temp = pd.DataFrame(result)
        df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(df_temp)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")

    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()

In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("누락된_상품ID_크롤링.csv")
숙소_id_list = df['상품ID'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/누락숙소.csv", index=False)
df_실패.to_csv("야놀자_크롤링/누락실패.csv", index=False)

df_정보.head()

## 2차 주중판매가 누락 상품ID 크롤링

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options)

In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))

In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_2차주중누락.csv',
                     failed_path='야놀자_크롤링/failed_2차주중누락.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result = []
    failed_ids = []
    is_first = not os.path.exists(result_path)

    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click()
            time.sleep(0.3)

            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                rating = driver.find_element(By.CLASS_NAME, 'typography-body-14-bold').text.strip()
            except:
                rating = "없음"

            try:
                review_count = driver.find_element(By.XPATH, '//p[@class="flex items-center justify-start gap-2 pl-2"]/span[2]').text.strip().replace("(", "").replace(")", "")
            except:
                review_count = "없음"

            주중판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주중판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except Exception as e:
                print(f" 숙박가 파싱 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주중판매가": 주중판매가
            })
            print(f" 수집결과 → {result[-1]}")

            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)

            except Exception as e:
                print(f"⚠️ 뒤로가기 실패 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)

            # 실패한 ID를 CSV에 저장
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path,
                mode='a',
                index=False,
                header=not os.path.exists(failed_path),
                encoding='utf-8-sig'
            )

            try:
                driver.quit()
            except:
                pass
            driver = init_driver()
            url = 'https://nol.yanolja.com/search?tab=place'
            driver.get(url)
            time.sleep(random.uniform(2.5, 4.0))
           
                       # ✅ 드라이버 재시작 시 날짜 재선택
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                        
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[5]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[6]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
                continue  # 날짜 클릭도 실패했으면 다음 ID로 넘김
           
            continue

        if len(result) >= save_every:
            df_temp = pd.DataFrame(result)
            df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(df_temp)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    if result:
        df_temp = pd.DataFrame(result)
        df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(df_temp)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")

    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()

In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("2차누락크롤링.csv")
숙소_id_list = df['숙소ID'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/2차누락숙소.csv", index=False)
df_실패.to_csv("야놀자_크롤링/2차누락실패.csv", index=False)

df_정보.head()

## 주말 가격

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options)

In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))


In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_total_주말.csv',
                     failed_path='야놀자_크롤링/failed_log_주말.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result = []
    failed_ids = []
    is_first = not os.path.exists(result_path)

    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click()
            time.sleep(0.3)

            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                rating = driver.find_element(By.CLASS_NAME, 'typography-body-14-bold').text.strip()
            except:
                rating = "없음"

            try:
                review_count = driver.find_element(By.XPATH, '//p[@class="flex items-center justify-start gap-2 pl-2"]/span[2]').text.strip().replace("(", "").replace(")", "")
            except:
                review_count = "없음"

            주말판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주말판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except Exception as e:
                print(f" 숙박가 파싱 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주말판매가": 주말판매가
            })
            print(f" 수집결과 → {result[-1]}")

            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)

            except Exception as e:
                print(f"⚠️ 뒤로가기 실패 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)

            # 실패한 ID를 CSV에 저장
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path,
                mode='a',
                index=False,
                header=not os.path.exists(failed_path),
                encoding='utf-8-sig'
            )

            try:
                driver.quit()
            except:
                pass
            driver = init_driver()
            url = 'https://nol.yanolja.com/search?tab=place'
            driver.get(url)
            time.sleep(random.uniform(2.5, 4.0))
           
                       # ✅ 드라이버 재시작 시 날짜 재선택
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[3]/td[7]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[4]/td[1]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
                continue  # 날짜 클릭도 실패했으면 다음 ID로 넘김
           
            continue

        if len(result) >= save_every:
            df_temp = pd.DataFrame(result)
            df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(df_temp)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    if result:
        df_temp = pd.DataFrame(result)
        df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(df_temp)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")

    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()

In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("메인프레임.csv")
숙소_id_list = df['LDGS_CD'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보_주말, df_실패_주말 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보_주말.to_csv("야놀자_크롤링/숙소정보_주말.csv", index=False)
df_실패_주말.to_csv("야놀자_크롤링/실패목록_주말.csv", index=False)

df_정보_주말.head()

## 코드 오류로 수정

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
import random




options = uc.ChromeOptions()
options.add_argument("--start-maximized")

options.add_argument("--disable-blink-features=AutomationControlled")

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options)

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)

time.sleep(random.uniform(2.5, 4.0))


In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_결과/results_total.csv',
                     failed_path='야놀자_결과/failed_log.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f"🔄 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result = []
    failed_ids = []
    is_first = not os.path.exists(result_path)

    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f"🎯 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            # ⛔ 드라이버 정상 작동 여부 점검
            if not driver.title or "야놀자" not in driver.title:
                raise Exception("⚠️ 드라이버 비정상 상태 감지 (title 없음 또는 야놀자 미포함)")

            # 🔍 검색창 클릭 및 입력
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click()
            time.sleep(0.3)
            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(random.uniform(2.5, 3.5))

            # 결과 유무 체크
            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f"❗ 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                rating = driver.find_element(By.CLASS_NAME, 'typography-body-14-bold').text.strip()
            except:
                rating = "없음"

            try:
                review_count = driver.find_element(By.XPATH, '//p[@class="flex items-center justify-start gap-2 pl-2"]/span[2]').text.strip().replace("(", "").replace(")", "")
            except:
                review_count = "없음"

            주중판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주중판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except Exception as e:
                print(f"❌ 숙박가 파싱 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주중판매가": 주중판매가
            })
            print(f"✅ 수집결과 → {result[-1]}")

            # 뒤로가기
            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(random.uniform(1.0, 1.5))

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)

            except Exception as e:
                print(f"⚠️ 뒤로가기 실패 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f"🛑 에러 발생 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)
            

            try:
                driver.quit()
            except:
                pass
            time.sleep(3)
            driver = init_driver()  # 외부에 정의된 함수여야 합니다.
            continue

        # 중간 저장
        if len(result) >= save_every:
            df_temp = pd.DataFrame(result)
            df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f"💾 {len(df_temp)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    # 남은 결과 저장
    if result:
        df_temp = pd.DataFrame(result)
        df_temp.to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f"💾 마지막 {len(df_temp)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print("🎉 전체 크롤링 완료.")



In [ ]:
# ✅ 메인프레임 불러오기
df = pd.read_csv("메인프레임.csv")
숙소_id_list = df['LDGS_CD'].astype(str).tolist()

# ✅ 크롤링 실행
df_정보 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

# ✅ 최종 저장
df_정보.to_csv("야놀자_결과/숙소정보.csv", index=False)
df_정보.head()


- 확인 용

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# ✅ 드라이버 초기화
def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options)

# ✅ 날짜 클릭 함수
def click_day(driver, aria_label):
    xpath = f'//td[@aria-label="{aria_label}"]'
    element = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, xpath)))
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
    time.sleep(0.5)

# ✅ 날짜 전체 설정 함수
def 날짜_선택(driver):
    try:
        # 날짜 버튼 클릭
        date_btn_xpath = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button'
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, date_btn_xpath))).click()
        time.sleep(1.0)

        # 스크롤 대상 찾기 (모달 내부)
        scroll_target_xpath = '//div[contains(@class, "DayPicker_transitionContainer_verticalScrollable")]'
        scrollable = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, scroll_target_xpath)))
        driver.execute_script("arguments[0].scrollBy(0, 300);", scrollable)
        time.sleep(1.0)

        # 날짜 클릭
        click_day(driver, "Choose 화요일, 2025년 8월 19일")
        click_day(driver, "Choose 수요일, 2025년 8월 20일")

        # 확인 클릭
        confirm_xpath = '//button[contains(text(), "확인")]'
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, confirm_xpath))).click()
        time.sleep(1.0)

        print("✅ 날짜 선택 완료")
    except Exception as e:
        print(f"❌ 날짜 선택 실패 → {e}")

# ✅ 실행
driver = init_driver()
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(random.uniform(2.5, 4.0))

날짜_선택(driver)



In [ ]:

def 야놀자_숙소정보_추출(driver, 숙소_id_list):
    result = []

    for 숙소_id in 숙소_id_list:
        print(f"\n▶ 숙소 ID {숙소_id} 검색 중...")
        try:
            # 검색창에 ID 입력
            search_input = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input')
            search_input.clear()
            search_input.send_keys(숙소_id)
            time.sleep(0.5)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            # 별점
            rating = driver.find_element(By.CLASS_NAME, 'typography-body-14-bold').text.strip()

            # 리뷰 수
            review_count_elem = driver.find_element(By.XPATH, '//p[@class="flex items-center justify-start gap-2 pl-2"]/span[2]')
            review_count = review_count_elem.text.strip().replace("(", "").replace(")", "")

            # 할인률
            discount = driver.find_element(By.CLASS_NAME, 'ml-4.text-text-neutral-main.typography-body-12-regular').text.strip()

            # 정가
            original_price = driver.find_element(By.CLASS_NAME, 'ml-2.text-text-neutral-sub.line-through.typography-body-12-regular').text.strip().replace(",", "").replace("원", "")

            # 판매가
            price = driver.find_element(By.CLASS_NAME, 'shrink-0.grow-0.text-text-neutral-main.typography-subtitle-18-bold').text.strip().replace(",", "").replace("원", "")

        except Exception as e:
            print(f"❌ 오류 발생 (ID: {숙소_id}) → {e}")
            rating = review_count = discount = original_price = price = "정보 없음"

        result.append({
            "숙소ID": 숙소_id,
            "별점": rating,
            "리뷰수": review_count,
            "할인률": discount,
            "정가": original_price,
            "판매가": price
        })

    return result



In [ ]:
# 1. 숙소 ID 리스트 정의
all_product_CD = ['1000114577']  # 다른 숙소 ID도 여러 개 추가 가능

# 2. 함수 호출
df_result = 야놀자_숙소정보_추출(driver, all_product_CD)

# 3. 출력 결과 확인
print(df_result)

# 야놀자 누락 가격 크롤링

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options, version_main=138)

In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))

In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_2차주중누락.csv',
                     failed_path='야놀자_크롤링/failed_2차주중누락.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result, failed_ids = [], []
    is_first = not os.path.exists(result_path)
    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click(); time.sleep(0.3)
            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # ⭐️ 별점 추출
            try:
                rating_candidates = soup.find_all("span", class_="typography-body-14-bold")
                rating = "없음"
                for span in rating_candidates:
                    text = span.text.strip()
                    if text.replace(".", "").isdigit():
                        rating = text
                        break
            except Exception as e:
                print(f"❌ 별점 파싱 실패: {e}")
                rating = "없음"

            # ⭐️ 리뷰 수 추출 (두 구조 모두 대응)
            try:
                review_count = "없음"
                
                # 1️⃣ 별점 옆 구조: <span>5.0</span> <span>(3)</span>
                rating_span = soup.find("span", class_="typography-body-14-bold")
                if rating_span:
                    p_tag = rating_span.find_parent("p")
                    span_tags = p_tag.find_all("span")
                    if len(span_tags) >= 2:
                        second_span_text = span_tags[1].text.strip()
                        if second_span_text.startswith("(") and second_span_text.endswith(")"):
                            review_count = second_span_text[1:-1]

                # 2️⃣ 후기 텍스트 구조: '후기' 문자열 포함한 span
                if review_count == "없음":
                    review_span = soup.find("span", string=lambda t: t and "후기" in t)
                    if review_span:
                        p_tag = review_span.find_parent("p")
                        span_tags = p_tag.find_all("span")
                        if len(span_tags) >= 2:
                            review_text = span_tags[1].text.strip()
                            review_count = review_text.replace("(", "").replace(")", "")
            except Exception as e:
                print(f"❌ 리뷰수 파싱 실패: {e}")
                review_count = "없음"

            # ✅ 주중판매가
            주중판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주중판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except: pass

            # ✅ 예약마감일 경우 '숙박' 블록 내 '판매가' 추출
            if 주중판매가 in ["없음", "예약마감"]:
                try:
                    div_blocks = soup.find_all("div", class_="mt-6 flex flex-wrap items-center justify-end")
                    for div in div_blocks:
                        if div.find("span", string=lambda t: t and "숙박" in t):
                            판매가_span = div.find("span", string=lambda t: t and "판매가" in t)
                            if 판매가_span:
                                주중판매가 = (
                                    판매가_span.text.strip()
                                    .replace("판매가", "")
                                    .replace(",", "")
                                    .replace("원~", "")
                                    .replace("원", "")
                                    .strip()
                                )
                                print(f"📌 예약마감 → '숙박' 블록 내 판매가 추출 성공 → {주중판매가}")
                                break
                except Exception as e:
                    print(f"📌 예약마감 → '숙박' 판매가 추출 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주중판매가": 주중판매가
            })
            print(f" 수집결과 → {result[-1]}")

            # ✅ 검색창 초기화 & 뒤로가기
            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)
            except Exception as e:
                print(f"⚠️ 뒤로가기 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig'
            )

            try: driver.quit()
            except: pass

            driver = init_driver()
            driver.get("https://nol.yanolja.com/search?tab=place")
            time.sleep(random.uniform(2.5, 4.0))
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                        
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[5]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[6]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
            continue

        # ✅ 중간 저장
        if len(result) >= save_every:
            pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(result)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    # ✅ 최종 저장
    if result:
        pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(result)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")
    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()


In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("야놀자_2차누락크롤링.csv")
숙소_id_list = df['숙소ID'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/2차누락숙소.csv", index=False)
df_실패.to_csv("야놀자_크롤링/2차누락실패.csv", index=False)

df_정보.head()

## 주중 누락 판매가 크롤링

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options, version_main=138)

In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))

In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_3차주중누락.csv',
                     failed_path='야놀자_크롤링/failed_3차주중누락.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result, failed_ids = [], []
    is_first = not os.path.exists(result_path)
    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click(); time.sleep(0.3)
            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # ⭐️ 별점 추출
            try:
                rating_candidates = soup.find_all("span", class_="typography-body-14-bold")
                rating = "없음"
                for span in rating_candidates:
                    text = span.text.strip()
                    if text.replace(".", "").isdigit():
                        rating = text
                        break
            except Exception as e:
                print(f"❌ 별점 파싱 실패: {e}")
                rating = "없음"

            # ⭐️ 리뷰 수 추출 (두 구조 모두 대응)
            try:
                review_count = "없음"
                
                # 1️⃣ 별점 옆 구조: <span>5.0</span> <span>(3)</span>
                rating_span = soup.find("span", class_="typography-body-14-bold")
                if rating_span:
                    p_tag = rating_span.find_parent("p")
                    span_tags = p_tag.find_all("span")
                    if len(span_tags) >= 2:
                        second_span_text = span_tags[1].text.strip()
                        if second_span_text.startswith("(") and second_span_text.endswith(")"):
                            review_count = second_span_text[1:-1]

                # 2️⃣ 후기 텍스트 구조: '후기' 문자열 포함한 span
                if review_count == "없음":
                    review_span = soup.find("span", string=lambda t: t and "후기" in t)
                    if review_span:
                        p_tag = review_span.find_parent("p")
                        span_tags = p_tag.find_all("span")
                        if len(span_tags) >= 2:
                            review_text = span_tags[1].text.strip()
                            review_count = review_text.replace("(", "").replace(")", "")
            except Exception as e:
                print(f"❌ 리뷰수 파싱 실패: {e}")
                review_count = "없음"

            # ✅ 주중판매가
            주중판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주중판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except: pass

            # ✅ 예약마감일 경우 '숙박' 블록 내 '판매가' 추출
            if 주중판매가 in ["없음", "예약마감"]:
                try:
                    div_blocks = soup.find_all("div", class_="mt-6 flex flex-wrap items-center justify-end")
                    for div in div_blocks:
                        if div.find("span", string=lambda t: t and "숙박" in t):
                            판매가_span = div.find("span", string=lambda t: t and "판매가" in t)
                            if 판매가_span:
                                주중판매가 = (
                                    판매가_span.text.strip()
                                    .replace("판매가", "")
                                    .replace(",", "")
                                    .replace("원~", "")
                                    .replace("원", "")
                                    .strip()
                                )
                                print(f"📌 예약마감 → '숙박' 블록 내 판매가 추출 성공 → {주중판매가}")
                                break
                except Exception as e:
                    print(f"📌 예약마감 → '숙박' 판매가 추출 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주중판매가": 주중판매가
            })
            print(f" 수집결과 → {result[-1]}")

            # ✅ 검색창 초기화 & 뒤로가기
            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)
            except Exception as e:
                print(f"⚠️ 뒤로가기 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig'
            )

            try: driver.quit()
            except: pass

            driver = init_driver()
            driver.get("https://nol.yanolja.com/search?tab=place")
            time.sleep(random.uniform(2.5, 4.0))
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                        
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[3]/td[2]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[3]/td[3]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
            continue

        # ✅ 중간 저장
        if len(result) >= save_every:
            pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(result)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    # ✅ 최종 저장
    if result:
        pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(result)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")
    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()

In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("누락_숙소ID_목록.csv")
숙소_id_list = df['숙소ID'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/3차누락숙소.csv", index=False)
df_실패.to_csv("야놀자_크롤링/3차누락실패.csv", index=False)

df_정보.head()

## 주말 누락숙소 크롤링

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options, version_main=138)

In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))

In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_1차주말누락.csv',
                     failed_path='야놀자_크롤링/failed_1차주말누락.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result, failed_ids = [], []
    is_first = not os.path.exists(result_path)
    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click(); time.sleep(0.3)
            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # ⭐️ 별점 추출
            try:
                rating_candidates = soup.find_all("span", class_="typography-body-14-bold")
                rating = "없음"
                for span in rating_candidates:
                    text = span.text.strip()
                    if text.replace(".", "").isdigit():
                        rating = text
                        break
            except Exception as e:
                print(f"❌ 별점 파싱 실패: {e}")
                rating = "없음"

            # ⭐️ 리뷰 수 추출 (두 구조 모두 대응)
            try:
                review_count = "없음"
                
                # 1️⃣ 별점 옆 구조: <span>5.0</span> <span>(3)</span>
                rating_span = soup.find("span", class_="typography-body-14-bold")
                if rating_span:
                    p_tag = rating_span.find_parent("p")
                    span_tags = p_tag.find_all("span")
                    if len(span_tags) >= 2:
                        second_span_text = span_tags[1].text.strip()
                        if second_span_text.startswith("(") and second_span_text.endswith(")"):
                            review_count = second_span_text[1:-1]

                # 2️⃣ 후기 텍스트 구조: '후기' 문자열 포함한 span
                if review_count == "없음":
                    review_span = soup.find("span", string=lambda t: t and "후기" in t)
                    if review_span:
                        p_tag = review_span.find_parent("p")
                        span_tags = p_tag.find_all("span")
                        if len(span_tags) >= 2:
                            review_text = span_tags[1].text.strip()
                            review_count = review_text.replace("(", "").replace(")", "")
            except Exception as e:
                print(f"❌ 리뷰수 파싱 실패: {e}")
                review_count = "없음"

            # ✅ 주중판매가
            주중판매가 = "없음"
            try:
                숙박_텍스트 = soup.find("span", string=lambda t: t and "숙박" in t)
                if 숙박_텍스트:
                    숙박_박스 = 숙박_텍스트.find_parent("div").find_next_sibling("div")
                    주중판매가 = 숙박_박스.text.strip().replace(",", "").replace("원~", "").replace("원", "")
            except: pass

            # ✅ 예약마감일 경우 '숙박' 블록 내 '판매가' 추출
            if 주중판매가 in ["없음", "예약마감"]:
                try:
                    div_blocks = soup.find_all("div", class_="mt-6 flex flex-wrap items-center justify-end")
                    for div in div_blocks:
                        if div.find("span", string=lambda t: t and "숙박" in t):
                            판매가_span = div.find("span", string=lambda t: t and "판매가" in t)
                            if 판매가_span:
                                주중판매가 = (
                                    판매가_span.text.strip()
                                    .replace("판매가", "")
                                    .replace(",", "")
                                    .replace("원~", "")
                                    .replace("원", "")
                                    .strip()
                                )
                                print(f"📌 예약마감 → '숙박' 블록 내 판매가 추출 성공 → {주중판매가}")
                                break
                except Exception as e:
                    print(f"📌 예약마감 → '숙박' 판매가 추출 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "별점": rating,
                "리뷰수": review_count,
                "주중판매가": 주중판매가
            })
            print(f" 수집결과 → {result[-1]}")

            # ✅ 검색창 초기화 & 뒤로가기
            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)
            except Exception as e:
                print(f"⚠️ 뒤로가기 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig'
            )

            try: driver.quit()
            except: pass

            driver = init_driver()
            driver.get("https://nol.yanolja.com/search?tab=place")
            time.sleep(random.uniform(2.5, 4.0))
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                        
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[7]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[3]/td[1]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
            continue

        # ✅ 중간 저장
        if len(result) >= save_every:
            pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(result)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    # ✅ 최종 저장
    if result:
        pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(result)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")
    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()

In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("야놀자_크롤링/0806주말_누락숙소.csv")
숙소_id_list = df['숙소ID'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/1차주말누락숙소.csv", index=False)
df_실패.to_csv("야놀자_크롤링/1차주말누락실패.csv", index=False)

df_정보.head()

## 비정상_주말판매가

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options, version_main=138)

In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))

In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_3차주말누락.csv',
                     failed_path='야놀자_크롤링/failed_3차주말누락.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result, failed_ids = [], []
    is_first = not os.path.exists(result_path)
    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click(); time.sleep(0.3)
            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # ✅ 주말판매가 추출 (두 번째 가격)
            주말판매가 = "없음"

            try:
                price_tags = soup.find_all("span", class_="shrink-0 grow-0 text-text-neutral-main typography-subtitle-18-bold")
                if len(price_tags) >= 2:
                    price_text = price_tags[1].text.strip()
                elif len(price_tags) == 1:
                    price_text = price_tags[0].text.strip()
                else:
                    price_text = "없음"

                주말판매가 = (
                    price_text.replace(",", "")
                              .replace("원~", "")
                              .replace("원", "")
                              .strip()
                )
            except Exception as e:
                print(f"❌ 주말판매가 파싱 실패: {e}")
                주말판매가 = "없음"

            # ✅ 예약마감 예외 처리
            if 주말판매가 in ["없음", "예약마감"]:
                try:
                    div_blocks = soup.find_all("div", class_="mt-6 flex flex-wrap items-center justify-end")
                    for div in div_blocks:
                        if div.find("span", string=lambda t: t and "숙박" in t):
                            판매가_span = div.find("span", string=lambda t: t and "판매가" in t)
                            if 판매가_span:
                                주말판매가 = (
                                    판매가_span.text.strip()
                                        .replace("판매가", "")
                                        .replace(",", "")
                                        .replace("원~", "")
                                        .replace("원", "")
                                        .strip()
                                )
                                print(f"📌 예약마감 → '숙박' 블록 내 판매가 추출 성공 → {주말판매가}")
                                break
                except Exception as e:
                    print(f"📌 예약마감 → '숙박' 판매가 추출 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "주말판매가": 주말판매가
            })
            print(f" 수집결과 → {result[-1]}")

            # ✅ 검색창 초기화 & 뒤로가기
            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)
            except Exception as e:
                print(f"⚠️ 뒤로가기 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig'
            )

            try: driver.quit()
            except: pass

            driver = init_driver()
            driver.get("https://nol.yanolja.com/search?tab=place")
            time.sleep(random.uniform(2.5, 4.0))
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                        
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[6]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[7]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
            continue

        # ✅ 중간 저장
        if len(result) >= save_every:
            pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(result)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    # ✅ 최종 저장
    if result:
        pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(result)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")
    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()


In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("야놀자_크롤링/비정상_주말판매가.csv")
숙소_id_list = df['숙소ID'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/1비정상_주말판매가.csv", index=False)
df_실패.to_csv("야놀자_크롤링/1비정상_주말판매가실패.csv", index=False)

df_정보.head()

## 비정상_주중판매가

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
    return uc.Chrome(options=options, version_main=138)

In [ ]:
driver = init_driver()

url = 'https://nol.yanolja.com/search?tab=place'
driver.get(url)
time.sleep(random.uniform(2.5, 4.0))

In [ ]:
def 야놀자_숙소정보_추출(driver,
                     숙소_id_list,
                     save_every=50,
                     result_path='야놀자_크롤링/results_3차주중누락.csv',
                     failed_path='야놀자_크롤링/failed_3차주중누락.csv'):

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    if os.path.exists(result_path):
        saved_df = pd.read_csv(result_path)
        saved_ids = set(saved_df['숙소ID'].astype(str).tolist())
        print(f" 기존 저장된 ID 수: {len(saved_ids)}")
    else:
        saved_ids = set()

    result, failed_ids = [], []
    is_first = not os.path.exists(result_path)
    target_ids = [str(i) for i in 숙소_id_list if str(i) not in saved_ids]
    print(f" 수집 대상 ID 수: {len(target_ids)}")

    for idx, 숙소_id in enumerate(target_ids):
        print(f"\n▶ ({idx+1}/{len(target_ids)}) 숙소 ID {숙소_id} 검색 중...")

        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
            )
            search_input.click(); time.sleep(0.3)
            search_input.clear()
            search_input.send_keys(숙소_id)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2.5)

            cards = driver.find_elements(By.CLASS_NAME, 'relative')
            if not cards:
                print(f" 숙소 ID {숙소_id} 결과 없음")
                failed_ids.append(숙소_id)
                continue

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # ✅ 주중판매가 추출 (두 번째 가격)
            주중판매가 = "없음"

            try:
                price_tags = soup.find_all("span", class_="shrink-0 grow-0 text-text-neutral-main typography-subtitle-18-bold")
                if len(price_tags) >= 2:
                    price_text = price_tags[1].text.strip()
                elif len(price_tags) == 1:
                    price_text = price_tags[0].text.strip()
                else:
                    price_text = "없음"

                주중판매가 = (
                    price_text.replace(",", "")
                              .replace("원~", "")
                              .replace("원", "")
                              .strip()
                )
            except Exception as e:
                print(f"❌ 주중판매가 파싱 실패: {e}")
                주중판매가 = "없음"

            # ✅ 예약마감 예외 처리
            if 주중판매가 in ["없음", "예약마감"]:
                try:
                    div_blocks = soup.find_all("div", class_="mt-6 flex flex-wrap items-center justify-end")
                    for div in div_blocks:
                        if div.find("span", string=lambda t: t and "숙박" in t):
                            판매가_span = div.find("span", string=lambda t: t and "판매가" in t)
                            if 판매가_span:
                                주중판매가 = (
                                    판매가_span.text.strip()
                                        .replace("판매가", "")
                                        .replace(",", "")
                                        .replace("원~", "")
                                        .replace("원", "")
                                        .strip()
                                )
                                print(f"📌 예약마감 → '숙박' 블록 내 판매가 추출 성공 → {주중판매가}")
                                break
                except Exception as e:
                    print(f"📌 예약마감 → '숙박' 판매가 추출 실패: {e}")

            result.append({
                "숙소ID": 숙소_id,
                "주중판매가": 주중판매가
            })
            print(f" 수집결과 → {result[-1]}")

            # ✅ 검색창 초기화 & 뒤로가기
            try:
                back_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Back" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", back_button)
                time.sleep(1.2)

                search_input = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/div/div[1]/form/div/div[1]/div/input'))
                )
                search_input.click()
                time.sleep(0.3)
            except Exception as e:
                print(f"⚠️ 뒤로가기 or 검색창 재활성화 실패 → {e}")

        except Exception as e:
            print(f" 전체 크롤링 실패 (ID: {숙소_id}) → {e}")
            failed_ids.append(숙소_id)
            pd.DataFrame({"숙소ID": [숙소_id]}).to_csv(
                failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig'
            )

            try: driver.quit()
            except: pass

            driver = init_driver()
            driver.get("https://nol.yanolja.com/search?tab=place")
            time.sleep(random.uniform(2.5, 4.0))
            try:
                driver.find_element(By.XPATH , value = '//*[@id="__next"]/div/main/div/div[1]/form/div/div[2]/div[1]/button').click()
                time.sleep(0.4)                        
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[6]').click()
                time.sleep(0.4)                         
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[3]/div/div/div/div[2]/div/div[2]/div[1]/div/table/tbody/tr[2]/td[7]').click()
                time.sleep(0.4)
                driver.find_element(By.XPATH , value = '/html/body/div[4]/div/div/section/section[4]/button').click()
                print("🔁 드라이버 재시작 후 날짜 재선택 완료")
            except Exception as e:
                print(f" 드라이버 재시작 후 날짜 재선택 실패 → {e}")
            continue

        # ✅ 중간 저장
        if len(result) >= save_every:
            pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
            print(f" {len(result)}개 저장 완료 → {result_path}")
            result = []
            is_first = False

    # ✅ 최종 저장
    if result:
        pd.DataFrame(result).to_csv(result_path, mode='a', header=is_first, index=False, encoding='utf-8-sig')
        print(f" 마지막 {len(result)}개 저장 완료 → {result_path}")

    if failed_ids:
        pd.DataFrame({"숙소ID": failed_ids}).to_csv(failed_path, mode='a', index=False, header=not os.path.exists(failed_path), encoding='utf-8-sig')
        print(f"⚠️ 실패 ID {len(failed_ids)}개 저장 완료 → {failed_path}")

    print(" 전체 크롤링 완료.")
    return pd.read_csv(result_path), pd.read_csv(failed_path) if os.path.exists(failed_path) else pd.DataFrame()


In [ ]:
#  메인프레임 불러오기
df = pd.read_csv("야놀자_크롤링/비정상_주중판매가.csv")
숙소_id_list = df['숙소ID'].astype(str).tolist()

#  크롤링 실행 (함수에서 결과 + 실패 반환)
df_정보, df_실패 = 야놀자_숙소정보_추출(driver, 숙소_id_list)

#  최종 저장
df_정보.to_csv("야놀자_크롤링/1비정상_주중판매가.csv", index=False)
df_실패.to_csv("야놀자_크롤링/1비정상_주중판매가실패.csv", index=False)

df_정보.head()

# 주말 판매가격 오류 수정

In [ ]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# === 1. 드라이버 설정 ===
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.169 Safari/537.36")
driver = uc.Chrome(options=options, version_main=138)

# === 2. 진입용 페이지 열기 ===
driver.get("https://nol.yanolja.com/search?tab=place")
time.sleep(0.4)

# === 3. 숙소 ID 불러오기 ===
df = pd.read_csv('0814_판매가비율1미만.csv')
# 숙소운영형태 Motel은 삭제
df = df[df['숙소운영형태'] != 'Motel']
ids = df['숙소ID'].astype(str).tolist()

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def 야놀자_주말판매가(driver, id_list):
    result = []

    for idx, pid in enumerate(id_list):
        print(f"\n[{idx+1}/{len(id_list)}] 숙소 ID {pid} 접속 중...")

        try:
            driver.get(f"https://place-site.yanolja.com/places/{pid}")
            time.sleep(2.5)

            # 스크롤 (필요한 경우만)
            for _ in range(3):
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(0.8)

            주말판매가 = "정보 없음"

            # 숙박 요금제 컨테이너 찾기
            containers = driver.find_elements(By.CLASS_NAME, "rate-plan-container")
            for c in containers:
                if "숙박" not in c.text:
                    continue

                try:
                    # 가격 스팬
                    price_el = c.find_element(By.CSS_SELECTOR, "span.price")
                    price_text = price_el.text.strip()

                    # '원' 단위 스팬 (있을 경우)
                    unit_text = ""
                    try:
                        unit_el = c.find_element(By.CSS_SELECTOR, "span.krw")
                        unit_text = unit_el.text.strip()
                    except:
                        pass

                    주말판매가 = price_text + unit_text

                    # ★ 1) 루프 내 가격 로그
                    print(f"   → 가격 캡처: {주말판매가}")

                    break  # 첫 번째 숙박 가격만 사용

                except Exception:
                    continue

        except Exception as e:
            print(f"❌ [접속 오류] 숙소 ID {pid}: {e}")
            주말판매가 = "접속 오류"

        # 결과 저장
        result.append({
            "숙소ID": pid,
            "주말판매가": 주말판매가
        })

        # ★ 1) 최종 로그
        print(f"[결과] {pid}: {주말판매가}")

    return pd.DataFrame(result)

# === 실행 ===
df_result = 야놀자_주말판매가(driver, ids)
df_result.to_csv("0814야놀자_주말판매가.csv", index=False)
df_result.head()


In [ ]:
df_result.to_csv("프로젝트/0814야놀자_주말판매가크롤링.csv", index=False)    

# 숙소 이미지 다운 

In [ ]:
import pandas as pd

df_image = pd.read_csv('이미지URL_최신3개.csv')

In [ ]:
df_image

In [ ]:
# 컬럼명 변경
df_image.rename(columns={'LDGS_CD' : '숙소ID', 'GSRM_IMAGE_URL' : '이미지URL'}, inplace=True)

In [ ]:
df_image

In [ ]:
import os
import requests
import pandas as pd

# 저장 폴더
save_dir = "야놀자ocr"
os.makedirs(save_dir, exist_ok=True)

df_test = df_image

# 숙소ID별 순번 카운터
counters = {}

# 실패 ID 목록
failed = []

total = len(df_test)
for i, row in enumerate(df_test.itertuples(), 1):
    sid = str(row.숙소ID)
    url = str(row.이미지URL).strip()

    # 진행상황 출력
    print(f"[{i}/{total}] {sid}")

    # URL이 비어있거나 NaN이면 실패 목록에 추가
    if not url or url.lower() == 'nan':
        failed.append((sid, url, 'empty_url'))
        continue

    counters[sid] = counters.get(sid, 0) + 1
    idx = counters[sid]
    save_path = os.path.join(save_dir, f"{sid}_{idx:02d}.jpg")

    try:
        res = requests.get(url, timeout=10)
        if res.status_code == 200 and 'image' in res.headers.get('Content-Type', ''):
            with open(save_path, 'wb') as f:
                f.write(res.content)
            print(f" {os.path.basename(save_path)} 저장 완료")
        else:
            failed.append((sid, url, f"status:{res.status_code}"))
    except Exception as e:
        failed.append((sid, url, f"error:{type(e).__name__}"))

# 실패 목록 CSV 저장 (이미지URL 포함)
if failed:
    pd.DataFrame(failed, columns=['숙소ID', '이미지URL', '사유']).to_csv('야놀자숙소이미지_실패1.csv', index=False, encoding='utf-8-sig')
    print(f"\n🚨 실패 ID {len(failed)}건 저장 완료: 야놀자숙소이미지_실패1.csv")
else:
    print("\n 모든 이미지 다운로드 성공")

In [ ]:
import os
import requests
import pandas as pd

# 저장 폴더
save_dir = "야놀자ocr"
os.makedirs(save_dir, exist_ok=True)

# ① 실패 목록 CSV 불러오기
df_failed = pd.read_csv("야놀자숙소이미지_실패1.csv", dtype={'숙소ID': str})

# 숙소ID별 순번 카운터
counters = {}

total = len(df_failed)
for i, row in enumerate(df_failed.itertuples(), 1):
    sid = str(row.숙소ID)
    url = str(row.이미지URL).strip()

    print(f"[{i}/{total}] {sid}")

    if not url or url.lower() == 'nan':
        continue

    counters[sid] = counters.get(sid, 0) + 1
    idx = counters[sid]
    save_path = os.path.join(save_dir, f"{sid}_{idx:02d}.jpg")

    try:
        res = requests.get(url, timeout=10)
        if res.status_code == 200 and 'image' in res.headers.get('Content-Type', ''):
            with open(save_path, 'wb') as f:
                f.write(res.content)
            print(f" {os.path.basename(save_path)} 저장 완료")
        else:
            print(f" ❌ 저장 실패: status:{res.status_code}")
    except Exception as e:
        print(f" ❌ 저장 실패: error:{type(e).__name__}")


In [ ]:
# 야놀자ocr 이미지 갯수 확인
image_files = [f for f in os.listdir(save_dir) if f.endswith('.jpg')]
print(f"야놀자ocr에 저장된 이미지 수: {len(image_files)}")


- 이미지 파일 누락값 비교

In [ ]:
import os
import pandas as pd

# 저장 폴더 경로
save_dir = "야놀자ocr"

# 저장된 이미지 파일명 리스트
image_files = [f for f in os.listdir(save_dir) if f.lower().endswith('.jpg')]
print(f"야놀자ocr에 저장된 이미지 수: {len(image_files)}")
print(f"df_image 행 수: {len(df_image)}")

# 저장된 숙소ID 추출
saved_ids = set([name.split('_')[0] for name in image_files])
all_ids = set(df_image['숙소ID'].astype(str))

# 누락된 숙소ID
missing_ids = all_ids - saved_ids
print(f"누락된 숙소ID 개수: {len(missing_ids)}")

# 누락된 숙소ID와 이미지URL 데이터프레임 생성
missing_df = df_image[df_image['숙소ID'].astype(str).isin(missing_ids)][['숙소ID', '이미지URL']].reset_index(drop=True)
missing_df.head()

In [ ]:
import os
import pandas as pd

save_dir = "야놀자ocr"
image_files = [f for f in os.listdir(save_dir) if f.lower().endswith('.jpg')]

# 1. df_image에서 {숙소ID: [이미지URL, ...]} dict 생성
id2urls = {}
for _, row in df_image.iterrows():
    sid = str(row['숙소ID'])
    id2urls.setdefault(sid, []).append(row['이미지URL'])

# 2. 저장된 파일명에서 바로 URL 추출
saved_urls = set()
for fname in image_files:
    sid, idx = fname.split('_')[0], int(fname.split('_')[1].split('.')[0])
    urls = id2urls.get(sid, [])
    if 0 < idx <= len(urls):
        saved_urls.add(urls[idx-1])

# 3. 누락된 이미지URL 찾기
all_urls = set(df_image['이미지URL'])
missing_urls = all_urls - saved_urls
print(f"누락된 이미지URL 개수: {len(missing_urls)}")

# 4. 누락된 숙소ID와 이미지URL 데이터프레임
missing_df = df_image[df_image['이미지URL'].isin(missing_urls)][['숙소ID', '이미지URL']].reset_index(drop=True)
missing_df.head()

- 154개 누락값 크롤링

In [ ]:
import os
import pandas as pd

save_dir = "야놀자ocr"
image_files = [f for f in os.listdir(save_dir) if f.lower().endswith('.jpg')]

# 1. 파일명에서 숙소ID별로 저장된 번호 리스트 추출
from collections import defaultdict
saved_idx_dict = defaultdict(set)
for fname in image_files:
    sid, idx = fname.split('_')[0], int(fname.split('_')[1].split('.')[0])
    saved_idx_dict[sid].add(idx)

# 2. df_failed에서 기대하는 번호 리스트 생성
expected_idx_dict = defaultdict(list)
for _, row in df_failed.iterrows():
    sid = str(row['숙소ID'])
    expected_idx_dict[sid].append(row['이미지URL'])

# 3. 누락된 번호 파악 및 누락 이미지URL 추출
missing_list = []
for sid, urls in expected_idx_dict.items():
    for i, url in enumerate(urls, 1):
        if i not in saved_idx_dict[sid]:
            missing_list.append((sid, i, url))

# 4. 누락된 숙소ID, 번호, 이미지URL 데이터프레임
missing_df = pd.DataFrame(missing_list, columns=['숙소ID', '번호', '이미지URL'])
print(f"누락된 이미지 개수: {len(missing_df)}")
missing_df.head()

In [ ]:
# missing_df에서 3014319호출
missing_3014319 = missing_df[missing_df['숙소ID'] == "3014319"]

In [ ]:
import os
import requests

save_dir = "야놀자ocr"
os.makedirs(save_dir, exist_ok=True)

# missing_3014319의 각 행에 대해 이미지 저장
for i, row in missing_3014319.iterrows():
    sid = str(row['숙소ID'])
    idx = int(row['번호']) if '번호' in missing_3014319.columns else i+1
    url = str(row['이미지URL']).strip()

    save_path = os.path.join(save_dir, f"{sid}_{idx:02d}.jpg")

    try:
        res = requests.get(url, timeout=10)
        if res.status_code == 200 and 'image' in res.headers.get('Content-Type', ''):
            with open(save_path, 'wb') as f:
                f.write(res.content)
            print(f"{os.path.basename(save_path)} 저장 완료")
        else:
            print(f"❌ 저장 실패: status:{res.status_code}")
    except Exception as e:
        print(f"❌ 저장 실패: error:{type(e).__name__}")

In [ ]:
import os
import math
import pandas as pd

save_dir = "야놀자ocr"
image_files = [f for f in os.listdir(save_dir) if f.lower().endswith('.jpg')]

# 3개로 균등 분할
n = len(image_files)
split_size = math.ceil(n / 3)
splits = [image_files[i*split_size:(i+1)*split_size] for i in range(3)]

# 파일로 저장
for idx, split in enumerate(splits, 1):
    df_split = pd.DataFrame(split, columns=['파일명'])
    out_path = f"야놀자ocr_split{idx}.csv"
    df_split.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"{out_path} 저장 완료: {len(df_split)}개")

In [ ]:
import os
import math
import shutil

save_dir = "야놀자ocr"
image_files = [f for f in os.listdir(save_dir) if f.lower().endswith('.jpg')]

# 3개로 균등 분할
n = len(image_files)
split_size = math.ceil(n / 3)
splits = [image_files[i*split_size:(i+1)*split_size] for i in range(3)]

# 폴더로 이동
for idx, split in enumerate(splits, 1):
    split_dir = f"야놀자ocr_part{idx}"
    os.makedirs(split_dir, exist_ok=True)
    for fname in split:
        src = os.path.join(save_dir, fname)
        dst = os.path.join(split_dir, fname)
        shutil.move(src, dst)
    print(f"{split_dir}로 {len(split)}개 이미지 이동 완료")

In [ ]:
# '_02.jpg'로 끝나는 이미지 파일만 추출하여 CSV로 저장
import os
import pandas as pd

save_dir = "야놀자ocr"
image_files = [f for f in os.listdir(save_dir) if f.lower().endswith('_02.jpg')]

# 파일명만 저장
df_02 = pd.DataFrame(image_files, columns=['파일명'])
df_02.to_csv("야놀자ocr_02only.csv", index=False, encoding='utf-8-sig')
print(f"야놀자ocr_02only.csv 저장 완료: {len(df_02)}개")


In [ ]:
# '_02.jpg'로 끝나는 이미지만 새로운 폴더로 복사
import os
import shutil

save_dir = "야놀자ocr"
target_dir = "야놀자ocr_02only"
os.makedirs(target_dir, exist_ok=True)

image_files = [f for f in os.listdir(save_dir) if f.lower().endswith('_02.jpg')]

for fname in image_files:
    src = os.path.join(save_dir, fname)
    dst = os.path.join(target_dir, fname)
    shutil.copy2(src, dst)

print(f"{target_dir} 폴더로 {len(image_files)}개 이미지 복사 완료")
